<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

# Debugging Models in TensorFlow <a class="tocSkip">

In [1]:
import os

import numpy as np

import tensorflow as tf
tf.set_random_seed(123)
print("TensorFlow:{}".format(tf.__version__))

TensorFlow:1.4.1


In [2]:
DATASETSLIB_HOME = '../datasetslib'
import sys
if not DATASETSLIB_HOME in sys.path:
    sys.path.append(DATASETSLIB_HOME)
%reload_ext autoreload
%autoreload 2
import datasetslib

In [3]:
datasetslib.datasets_root = os.path.join(os.path.expanduser('~'),'datasets')

# MNIST Dataset

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(os.path.join(datasetslib.datasets_root,'mnist'), one_hot=True)

x_train = mnist.train.images
x_test = mnist.test.images
y_train = mnist.train.labels
y_test = mnist.test.labels

# parameters
n_y = 10  # 0-9 digits
n_x = 784  # total pixels

Extracting /home/armando/datasets/mnist/train-images-idx3-ubyte.gz
Extracting /home/armando/datasets/mnist/train-labels-idx1-ubyte.gz
Extracting /home/armando/datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/armando/datasets/mnist/t10k-labels-idx1-ubyte.gz


# tf.Print()

In [5]:
def mlp(x, num_inputs, num_outputs,num_layers,num_neurons):
    w=[]
    b=[]
    for i in range(num_layers):
        # weights
        w.append(tf.Variable(tf.random_normal( 
                              [num_inputs if i==0 else num_neurons[i-1], 
                               num_neurons[i]]), 
                             name="w_{0:04d}".format(i) 
                            ) 
                ) 
        # biases
        b.append(tf.Variable(tf.random_normal( 
                              [num_neurons[i]]), 
                             name="b_{0:04d}".format(i) 
                            ) 
                )                   
    w.append(tf.Variable(tf.random_normal(
                          [num_neurons[num_layers-1] if num_layers > 0 else num_inputs,
                           num_outputs]),name="w_out"))
    b.append(tf.Variable(tf.random_normal([num_outputs]),name="b_out"))
    
    # x is input layer
    layer = x
    # add hidden layers
    for i in range(num_layers):
        layer = tf.nn.relu(tf.matmul(layer, w[i]) + b[i])
    # add output layer
    layer = tf.matmul(layer, w[num_layers]) + b[num_layers]
    
    return layer

In [6]:
tf.reset_default_graph()

In [7]:
num_layers = 2
num_neurons = [16,32]
learning_rate = 0.01
n_epochs = 10
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)

# input images
x_p = tf.placeholder(dtype=tf.float32, name="x_p", shape=[None, n_x]) 
# target output
y_p = tf.placeholder(dtype=tf.float32, name="y_p", shape=[None, n_y]) 

model = mlp(x=x_p, 
            num_inputs=n_x, 
            num_outputs=n_y, 
            num_layers=num_layers, 
            num_neurons=num_neurons)

model = tf.Print(input_=model,
                 data=[tf.argmax(model,1)],
                 message='y_hat=',
                 summarize=10,
                 first_n=5
                )

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y_p))
# optimizer function
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = optimizer.minimize(loss)

#predictions_check = tf.equal(tf.argmax(model,1), tf.argmax(y,1))
#accuracy_function = tf.reduce_mean(tf.cast(predictions_check, tf.float32))

with tf.Session() as tfs:
        tfs.run(tf.global_variables_initializer())
        for epoch in range(n_epochs):
            epoch_loss = 0.0
            for batch in range(n_batches):
                X_batch, Y_batch = mnist.train.next_batch(batch_size)
                feed_dict={x_p: X_batch, y_p: Y_batch}
                _,batch_loss = tfs.run([optimizer,loss], 
                                       feed_dict = feed_dict
                                      )
                epoch_loss += batch_loss 
            average_loss = epoch_loss / n_batches
            print("epoch: {0:04d}   loss = {1:0.6f}".format(epoch,average_loss))

epoch: 0000   loss = 6.716061
epoch: 0001   loss = 2.002921
epoch: 0002   loss = 1.733159
epoch: 0003   loss = 1.596843
epoch: 0004   loss = 1.504387
epoch: 0005   loss = 1.435767
epoch: 0006   loss = 1.378511
epoch: 0007   loss = 1.329088
epoch: 0008   loss = 1.287037
epoch: 0009   loss = 1.249800


# tf.Assert()

In [8]:
def mlp(x, num_inputs, num_outputs,num_layers,num_neurons):
    w=[]
    b=[]
    for i in range(num_layers):
        # weights
        w.append(tf.Variable(tf.random_normal( 
                              [num_inputs if i==0 else num_neurons[i-1], 
                               num_neurons[i]]), 
                             name="w_{0:04d}".format(i) 
                            ) 
                ) 
        # biases
        b.append(tf.Variable(tf.random_normal( 
                              [num_neurons[i]]), 
                             name="b_{0:04d}".format(i) 
                            ) 
                )                   
    w.append(tf.Variable(tf.random_normal(
                          [num_neurons[num_layers-1] if num_layers > 0 else num_inputs,
                           num_outputs]),name="w_out"))
    b.append(tf.Variable(tf.random_normal([num_outputs]),name="b_out"))
    
    assert_op = tf.Assert(tf.reduce_all(tf.greater_equal(x,0)),[x])
    with tf.control_dependencies([assert_op]):
        # x is input layer
        layer = x
        # add hidden layers
        for i in range(num_layers):
            layer = tf.nn.relu(tf.matmul(layer, w[i]) + b[i])
        # add output layer
        layer = tf.matmul(layer, w[num_layers]) + b[num_layers]
    
    return layer

In [9]:
tf.reset_default_graph()

In [10]:
num_layers = 2
num_neurons = [16,32]
learning_rate = 0.01
n_epochs = 10
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)

# input images
x_p = tf.placeholder(dtype=tf.float32, name="x_p", shape=[None, n_x]) 
# target output
y_p = tf.placeholder(dtype=tf.float32, name="y_p", shape=[None, n_y]) 

model = mlp(x=x_p, 
            num_inputs=n_x, 
            num_outputs=n_y, 
            num_layers=num_layers, 
            num_neurons=num_neurons)

model = tf.Print(input_=model,
                 data=[tf.argmax(model,1)],
                 message='y_hat=',
                 summarize=10,
                 first_n=5
                )

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y_p))
# optimizer function
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = optimizer.minimize(loss)

#predictions_check = tf.equal(tf.argmax(model,1), tf.argmax(y,1))
#accuracy_function = tf.reduce_mean(tf.cast(predictions_check, tf.float32))

with tf.Session() as tfs:
        tfs.run(tf.global_variables_initializer())
        for epoch in range(n_epochs):
            epoch_loss = 0.0
            for batch in range(n_batches):
                X_batch, Y_batch = mnist.train.next_batch(batch_size)
                if epoch > 5:
                    X_batch = np.copy(X_batch)
                    X_batch[0,0]=-2
                feed_dict={x_p: X_batch, y_p: Y_batch}
                _,batch_loss = tfs.run([optimizer,loss], 
                                       feed_dict = feed_dict
                                      )
                epoch_loss += batch_loss 
            average_loss = epoch_loss / n_batches
            print("epoch: {0:04d}   loss = {1:0.6f}".format(epoch,average_loss))

epoch: 0000   loss = 7.803496
epoch: 0001   loss = 2.405806
epoch: 0002   loss = 2.057570
epoch: 0003   loss = 1.884273
epoch: 0004   loss = 1.768692
epoch: 0005   loss = 1.672444


InvalidArgumentError: assertion failed: [[-2 0 0]...]
	 [[Node: Assert/AssertGuard/Assert = Assert[T=[DT_FLOAT], summarize=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Assert/AssertGuard/Assert/Switch/_7, Assert/AssertGuard/Assert/Switch_1)]]
	 [[Node: Assert/AssertGuard/Assert/_10 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_18_Assert/AssertGuard/Assert", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'Assert/AssertGuard/Assert', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3